In [1]:
import pandas as pd
import numpy as np
import sys
from math import sqrt
sys.path.append('../..')
from modules import utils
import gpflow
from gpflow import set_trainable
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-07-07 00:53:04.628133: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 00:53:04.704387: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-07 00:53:04.706649: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-07 00:53:04.706661: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [ ]:
seed = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)
gpflow.config.set_default_seed(SEED)

In [2]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [3]:
jinja_df.tail()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
16827,"Mpumudde, Jinja",0.463428,33.206262,Jinja,2021-11-29 19:00:00+00:00,37.4454,47.906395,57.335814,47.493504,615c878d580358002ae96dc2,1379965,aq_g532
16828,"Mpumudde, Jinja",0.463428,33.206262,Jinja,2021-11-29 20:00:00+00:00,57.2599,61.122143,73.658571,61.470257,615c878d580358002ae96dc2,1379965,aq_g532
16829,"Mpumudde, Jinja",0.463428,33.206262,Jinja,2021-11-29 21:00:00+00:00,27.6905,40.096667,48.702381,38.925092,615c878d580358002ae96dc2,1379965,aq_g532
16830,"Mpumudde, Jinja",0.463428,33.206262,Jinja,2021-11-29 22:00:00+00:00,17.8444,32.145714,36.381786,30.198896,615c878d580358002ae96dc2,1379965,aq_g532
16831,"Mpumudde, Jinja",0.463428,33.206262,Jinja,2021-11-29 23:00:00+00:00,18.1647,32.135000,36.750000,30.314966,615c878d580358002ae96dc2,1379965,aq_g532


In [4]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [5]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


In [6]:
def cross_validation(final_df, idx, kernel_variance, lengthscales, likelihood_variance, trainable_kernel, 
                     trainable_variance, trainable_lengthscales):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train).reshape(-1, 1)
    if X_train.shape[0] > 9999:
        X_train = X_train[::2, :]
        y_train = y_train[::2, :]
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test).reshape(-1, 1)
    #to delete
    #X_train, y_train, X_test, y_test = X_train[:100, :], y_train[:100, :], X_test[:100, :], y_test[:100, :]
    
    if lengthscales == 'train_shape':
        lengthscales = np.ones(X_train.shape[1])
    
    if (lengthscales is None) & (kernel_variance is None):
        k = gpflow.kernels.RBF() + gpflow.kernels.Bias()
    elif lengthscales is None:
        k = gpflow.kernels.RBF(variance=kernel_variance) + gpflow.kernels.Bias()
    elif kernel_variance is None:
        k = gpflow.kernels.RBF(lengthscales=lengthscales) + gpflow.kernels.Bias()
    else:
        k = gpflow.kernels.RBF(lengthscales=lengthscales, variance=kernel_variance) + gpflow.kernels.Bias()
        
    m = gpflow.models.GPR(data=(X_train, y_train), kernel=k, mean_function=None)
    if likelihood_variance is None:
        pass
    else:
        m.likelihood.variance.assign(likelihood_variance)
    set_trainable(m.kernel.kernels[0].variance, trainable_kernel)
    set_trainable(m.likelihood.variance, trainable_variance)
    set_trainable(m.kernel.kernels[0].lengthscales, trainable_lengthscales)
    
    #optimization
    opt = gpflow.optimizers.Scipy()
    def objective_closure():
        return - m.log_marginal_likelihood()
    
    opt_logs = opt.minimize(objective_closure,
                            m.trainable_variables,
                            options=dict(maxiter=100))

    #prediction
    mean, var = m.predict_f(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, mean.numpy()))
    mape = mean_absolute_percentage_error(y_test, mean.numpy())
    return rmse, mape
    
#     return mean.numpy(), var.numpy(), Xtest, Ytest, round(rmse, 2)

#### The real work

In [7]:
# lengthscales = [[0.008, 0.008, 2], None, 'train_shape']
lengthscale = [0.008, 0.008, 2]
# likelihood_variances = [400, 625, None]
likelihood_variance = 400
# kernel_variances = [400, 625, None]
kernel_variance = None
# trainable_kernels = [False, True]
trainable_kernel = True 
# trainable_variances = [False, True]
trainable_variance = False
# trainable_lengthscales = [False, True]
trainable_lengthscale = True

In [8]:
# results_df = pd.DataFrame()
# count = 12
# results_df = pd.read_csv(f'../results/basic_results_{count}.csv')
# for lengthscale in lengthscales:
#     for likelihood_variance in likelihood_variances:
#         for kernel_variance in kernel_variances:
#             for trainable_kernel in trainable_kernels:
#                 for trainable_variance in trainable_variances:
#                     for trainable_lengthscale in trainable_lengthscales:
# #                         count+=1
rmse_list, mape_list = [], []
# print(f'EXPERIMENT {count}')
for i in range(len(latitudes)):
    try:
        rmse, mape = cross_validation(final_df, i, kernel_variance, lengthscale, likelihood_variance, 
                                trainable_kernel, trainable_variance, trainable_lengthscale)
        rmse_list.append(rmse)
        mape_list.append(mape)
        print(f'{device_ids[i]} successful')
    except Exception as e:
        print(e)
        print(f'{device_ids[i]} failed')
        break

mean_rmse = np.mean(rmse_list)
mean_mape = np.mean(mape_list)
results_dict= {'lengthscale':lengthscale, 'likelihood_variance':likelihood_variance, 
               'kernel_variance':kernel_variance, 'trainable_kernel':trainable_kernel, 
               'trainable_variance':trainable_variance, 'trainable_lengthscale':trainable_lengthscale, 
               'avg_rmse':mean_rmse, 'rmse_list':rmse_list, 'avg_mape':mean_mape, 'mape_list':mape_list}
print(results_dict)
results_df = results_df.append(results_dict, ignore_index=True)
results_df.to_csv(f'../results/basic_results_with_mape.csv', index=False)               

2023-07-07 00:53:07.928656: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-07 00:53:07.928683: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-07-07 00:53:07.928687: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-07-07 00:53:07.928801: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-07-07 00:53:07.928820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.116.04  Release Build  (dvs-builder@U16-I3-A16-3-3)  Thu Apr 27 18:11:06 UTC 2023
GCC version:  gcc 

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
689753 successful
1014698 successful
1014692 successful
1014697 successful
1290038 successful
1373035 successful
1373037 successful


2023-07-07 02:32:15.073269: W tensorflow/core/kernels/linalg/cholesky_op.cc:56] Cholesky decomposition was not successful. Eigen::LLT failed with error code 1. Filling lower-triangular output with NaNs.


1373038 successful
1373039 successful
1379965 successful
{'lengthscale': [0.008, 0.008, 2], 'likelihood_variance': 400, 'kernel_variance': None, 'trainable_kernel': True, 'trainable_variance': False, 'trainable_lengthscale': True, 'avg_rmse': 17.602283913305843, 'rmse_list': [11.357703684120983, 21.784479591116558, 10.642979615491374, 16.137419666626414, 14.217015350327113, 22.683998950042604, 16.83702882017617, 13.74363949037157, 24.066536477463124, 24.552037487322494], 'avg_mape': 0.4333220973865197, 'mape_list': [0.7324530404566082, 0.2928445948231008, 0.39467173636965136, 0.25240960334489276, 0.48885564676035914, 0.4498863473569986, 0.3469852721580636, 0.5960556637184437, 0.4387343836518835, 0.3403246852251957]}


NameError: name 'results_df' is not defined

In [10]:
results_df=pd.DataFrame()
results_df = results_df.append(results_dict, ignore_index=True)
# results_df.to_csv(f'../results/basic_results_with_mape.csv', index=False)               

In [11]:
rmse_list

[11.357703684120983,
 21.784479591116558,
 10.642979615491374,
 16.137419666626414,
 14.217015350327113,
 22.683998950042604,
 16.83702882017617,
 13.74363949037157,
 24.066536477463124,
 24.552037487322494]

In [12]:
mean_rmse

17.602283913305843

In [13]:
mape_list

[0.7324530404566082,
 0.2928445948231008,
 0.39467173636965136,
 0.25240960334489276,
 0.48885564676035914,
 0.4498863473569986,
 0.3469852721580636,
 0.5960556637184437,
 0.4387343836518835,
 0.3403246852251957]

In [14]:
mean_mape

0.4333220973865197